PRACTICA 3
TAREA 1


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('word2vec_sample')
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize
import gensim
import numpy as np
from nltk.data import find
import statistics
import itertools
import scipy.spatial as s

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [ ]:
stopwords = set(sw.words("english"))
def leskD(word, phrase):
  senses = wn.synsets(word)
  best_sense = senses[0]
  max_ovelap = 0
  word_tokens = set(word_tokenize(phrase))
  context = []
  for word in word_tokens:
    if word not in stopwords:
      context.append(word)

  for sense in senses:
    exam = word_tokenize(sense.definition())
    for ex in sense.examples():
      exam += word_tokenize(ex)
    signature = []
    for y in exam:
      if y not in stopwords:
        signature.append(y)
    overlap = len(list(set(signature).intersection(context)))
    if overlap > max_ovelap:
      max_ovelap = overlap
      best_sense = sense

  print(best_sense.definition())
  return best_sense

In [ ]:
print(leskD("bank", "The van pulled up outside the bank and three masked men got out."))

sloping land (especially the slope beside a body of water)
Synset('bank.n.01')


In [ ]:
def dCos(x,y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

def leskC(word, phrase):
  word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
  model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)
  v_mean =  model.vectors.mean(axis=0)
  d=[]
  dm=[]
  context = []
  senses = wn.synsets(word)
  best_sense = senses[0]
  max_ovelap = 0
  word_tokens = set(word_tokenize(phrase))

  for word in word_tokens:
    if word not in stopwords:
      if word in model.vocab:
        context.append(model[word])
      else:
        context.append(v_mean)

  v_cont = []
  for column in range(len(context[0])):
    aux = 0
    for row in range(len(context)):
      aux += context[row][column]
    #v_cont.append(aux)
    v_cont.append(aux/len(context[0]))
  
  for sense in senses:
    exam = set(word_tokenize(sense.definition() + " "+ " ".join(sense.examples()))).difference(stopwords)
    signature=[]
    for word in exam:
      if word in model.vocab:
        signature.append(model[word])
      else:
        signature.append(v_mean)
    v_sig = []
    for column in range(len(signature[0])):
      aux = 0
      for row in range(len(signature)):
        aux += signature[row][column]
      #v_sig.append(aux)
      v_sig.append(aux/len(signature[0]))
    d.append((sense, dCos(v_cont, v_sig)))
  return sorted(d, key=lambda t:t[1], reverse=True)

In [ ]:
sentido = leskC("bass", "I went fishing for some sea bass.")
#sentido_t = sentido[0].definition()
#print(sentido, sentido_t)
print(sentido)
print(sentido[0][0])
print(sentido[0][0].definition())

[(Synset('sea_bass.n.01'), 0.5321858971292431), (Synset('bass.n.08'), 0.5309086217649079), (Synset('freshwater_bass.n.01'), 0.4889060061784287), (Synset('bass.s.01'), 0.45519594654207735), (Synset('bass.n.02'), 0.346013269342772), (Synset('bass.n.06'), 0.3134628391845537), (Synset('bass.n.07'), 0.30911920173610075), (Synset('bass.n.03'), 0.28831348191634626), (Synset('bass.n.01'), 0.2786361774388861)]
Synset('sea_bass.n.01')
the lean flesh of a saltwater fish of the family Serranidae
